# Speed up inference

Fast inference was an important element of the similar competition
[CZII](https://www.kaggle.com/competitions/czii-cryo-et-object-identification/overview). 


**Preprocess on GPU:**

Preprocessing on CPU (resize and normalize) can take significant time comparable to model inference, but become negligible if you do it on GPU.

For 20 training tomo_ids with one model, the inference time is:

```text
240 sec CPU preprocessing
 65 sec GPU
```


**Use amp with T4×2:**

T4×2 is faster than P100×1 for float32 only marginary, but
automatic mixed precision (amp) speeds up by a factor of 2 on T4, while not on P100.

```text
160 sec  P100 float32 num_workers=2
135 sec  T4×2 float32 num_workers=1
 65 sec  T4×2 amp     num_workers=1
```

(20 trian tomo_ids, one model)


I am also interested in:

* TensorRT
* Test-time augmentation (TTA)
  
but not in this notebook (yet?).

```text
Local CV:
  0.8164 ± 0.0355
  5 folds [0.7612 0.8076 0.8449 0.8495 0.8186]

Public LB:
  Fold 0       0.583   66 min
  5-fold mean  0.665  227 min
```

**Version: 2**

Fix wrong y_pred in predict(), thanks to [sacuscreed's comment](https://www.kaggle.com/code/junkoda/speed-up-inference/comments#3170678),
but the Public LB 0.665 remains the same .

In [5]:
import numpy as np
import pandas as pd
import time
import argparse
from pathlib import Path

from tqdm.auto import tqdm
from torch.utils.data import DataLoader, default_collate
from PIL import Image

import torch
import torch.nn as nn
import torchvision.transforms.functional as TTF
import timm
import yaml

import multiprocessing as mp
from queue import Empty


INPUT_PATH = Path('/kaggle/input/byu-locating-bacterial-flagellar-motors-2025')
MODEL_PATH = Path('/kaggle/input/bacterial-public/weights/object/baseline')

# Data

In [8]:
def get_tomos(input_path: Path, data_type: str,*, n=None) -> list[Path]:
    """
    Args
      input_path (Path): Kaggle input directory
      data_type (str):   train or test
      n (Optional[int]): Use only first n train tomos (not applicable to test)
    """
    data_path = input_path / data_type
    tomo_paths = sorted(data_path.glob('*'))

    if (n is not None) and (data_type == 'train'):
        tomo_paths = tomo_paths[:n]
    
    return tomo_paths
    

def preprocess(img: torch.Tensor) -> torch.Tensor:
    """
    Resize and normalize

    Arg:
      img (Tensor[uint8]):   (batch_size, C, H, W)

    Returns:
      img (Tensor[float32]): (batch_size, C, size, size); size = 640
    """
    size = (640, 640)

    img = img.to(dtype=torch.float32)
    img = TTF.resize(img, size)  # (batch_size, C, size, size)

    batch_size, nch, h, w = img.shape
    q = torch.Tensor([0.05, 0.95]).to(img.device)
    x_min, x_max = torch.quantile(img.view(batch_size, nch * h * w), q, dim=1)
    x_min = x_min.view(batch_size, 1, 1, 1)
    x_max = x_max.view(batch_size, 1, 1, 1)

    img = (img - x_min) / (x_max - x_min)
    img = torch.clamp(img, 0, 1)

    return img


class Dataset(torch.utils.data.Dataset):
    """
    dataset = Dataset(tomo_path)
    
    Args:
      tomo_path (Path): directory including jpg images    
    """
    def __init__(self, tomo_path: Path):
        self.filenames = sorted(tomo_path.glob('*'))  # list[Path]

    def __len__(self) -> int:
        return len(self.filenames)

    def __getitem__(self, i: int) -> dict:
        filename = self.filenames[i]  # Path
        filebase = filename.stem
        assert filebase[:6] == 'slice_'
        slice_number = int(filebase[6:])  # slice_0000 -> int(0000)

        # Load, resize and normalize image
        img = Image.open(filename)
        W, H = img.size
        img = np.expand_dims(np.array(img), axis=0)  # array[uint8] (1, H, W)

        ret = {'img': img,
               'slice_number': slice_number,
               'shape': np.array((H, W), dtype=int),  # original shape H, W
        }

        return ret

    def loader(self, batch_size: int, num_workers: int):
        loader = DataLoader(self, batch_size=batch_size, num_workers=num_workers)
        return loader


# Model

The model is a simple 2D object detection using convnext_tiny.

The main theme of this notebook is inference,
but if you are intersted in this model,
see github:

https://github.com/junkoda/kaggle_bacterial_public

I followed sharifi76's great notebook:

https://www.kaggle.com/code/sharifi76/eda-visualization-yolov8

using only ±4 positive slices around the label for training.

Usually using all data is best in terms of score, but you can speedup training significantly by reducing negative samples with little loss in score. I haven't tried yet but you can probably improve score by adding negative samples and adjust threshold for no moters (and also with more augmentations).


* Model input is 640x640 image
* Pretrained model outputs 20x20 embedding vectors
* Predict 20x20
    + probabilty including target
    + offset within the coarse grid
* Object detection models also predict sizes of bounding boxes, but this model only predicts the center points.


In [9]:
import torch
import torch.nn as nn
import timm


class Model(nn.Module):
    def __init__(self, cfg_model: dict, *, pretrained=True, verbose=True):
        super().__init__()

        # Timm encoder
        name = cfg_model['encoder']
        in_channels = 1
        out_channels = 1

        self.encoder = timm.create_model(name, in_chans=in_channels, features_only=True, pretrained=pretrained)
        encoder_channels = self.encoder.feature_info.channels()
        self.segmentation_head = nn.Conv2d(encoder_channels[-1], out_channels, kernel_size=3, padding=1)
        self.regression_head = nn.Conv2d(encoder_channels[-1], out_channels=2, kernel_size=3, padding=1)
        self.criterion_seg = nn.BCEWithLogitsLoss()
        self.criterion_reg = nn.MSELoss()

        if verbose:
            print(name)

    def forward(self, img: torch.Tensor):
        """
        img (Tensor): (batch_size, 1, H, W)

        h, w = H // 32, W // 32
        """
        features = self.encoder(img)
        out = features[-1]  # (batch_size, embed_dim, h, w)
        y_pred = self.segmentation_head(out)  # (batch_size, 1, h, w)
        t_pred = self.regression_head(out)    # (batch_size, 2, h, w)

        return y_pred, t_pred


# Predict

In [10]:
def predict(tomo_path: Path,
            models: list[nn.Module],
            cfg: dict) -> dict:
    """
    Predict moter coordinate for one tomo_id
    At most one moter per tomo_id in test
    
    Args:
      model (nn.Module): Pytorch model
      dataset (Dataset): for one tomo_id
    """
    assert len(models) > 0
    tomo_id = tomo_path.name
    batch_size = cfg['batch_size']
    num_workers = cfg['num_workers']
    use_amp = cfg['use_amp']
    preprocess_device = cfg['preprocess_device']
    assert preprocess_device in ['cuda', 'cpu']

    dataset = Dataset(tomo_path)
    loader = dataset.loader(batch_size=batch_size, num_workers=num_workers)
    
    device = next(models[0].parameters()).device

    # Loop over all slices in one tomo_id
    best = (0, None)
    for d in loader:
        # Input image (batch_size, 1, H, W)
        if preprocess_device == 'cuda':
            img = d['img'].to(device) 
            img = preprocess(img)
        elif preprocess_device == 'cpu':
            img = preprocess(d['img'])
            img = img.to(device)  # input image (batch_size, 1, H, W)
        else:
            raise ValueError(preprocess_device)

        y_pred_sum, t_pred_sum = None, None
        for model in models:
            with torch.no_grad():
                with torch.amp.autocast(device_type='cuda',
                                        enabled=use_amp,
                                        dtype=torch.float16):
                    y_pred, t_pred = model(img) 

            y_pred = y_pred.sigmoid()  # (batch_size, 1, h, w)

            if y_pred_sum is None:
                y_pred_sum = y_pred
                t_pred_sum = t_pred
            else:
                y_pred_sum += y_pred
                t_pred_sum += t_pred

        y_pred_max = y_pred_sum.max().item() / len(models)
        del y_pred, t_pred

        # Keep most probable coordinate
        if y_pred_max > best[0]:
            bs, _, h, w = y_pred_sum.shape
        
            argmax = torch.unravel_index(y_pred_sum.argmax(), y_pred_sum.shape)  # b, ch, iy, ix
            i, _, iy, ix = [t.item() for t in argmax]
            slice_number = d['slice_number'][i].item()
            offset = t_pred_sum[i, :, iy, ix].cpu().numpy() / len(models)  # (2, )

            # Compute coodinate in original pixels
            H, W = d['shape'][i].numpy()    # Original image size
            x = (ix + offset[0]) * (W / w)
            y = (iy + offset[1]) * (H / h)
            
            best = (y_pred_max, slice_number, y, x)

    assert best[1] is not None

    # Return prediction
    n_slices = len(dataset.filenames)
    pred = {'tomo_id': tomo_id,
            'n_slices': n_slices,
            'y_pred': best[0],
            'zyx': best[1:]}
    return pred


# Submit

In [11]:
def create_submission(preds: list, th: float, ofilename: str) -> pd.DataFrame:
    """
    Args:
      preds (list[dict]): predictions
      th (float): threshold between no or one moter
      ofilename (str): submission.csv
    """
    rows = []
    count_positive = 0
    for pred in preds:
        if pred['y_pred'] < th:
            zyx = (-1, -1, -1)
        else:
            count_positive += 1
            zyx = pred['zyx']

        row = {'tomo_id': pred['tomo_id'],
               'Motor axis 0': zyx[0],
               'Motor axis 1': zyx[1],
               'Motor axis 2': zyx[2]}
        rows.append(row)

    submit = pd.DataFrame(rows)
    submit.to_csv(ofilename, float_format='%.8e', index=False)

    print('Submit %s: %d positives / %d tomo_ids' % (ofilename, count_positive, len(rows)))

    return submit

# Main

In [12]:
def process_fn(process_id: int,
               tomo_queue,
               pred_queue,
               cfg: dict):
    """
    Prediction process for each GPU

    Args:
      process_id (int): 0 or 1 for 2 GPUs
      tomo_queue (): tomo_ids
      pred_queue (): output predictions
      cfg (dict): config
    """
    device = torch.device('cuda:%d' % process_id)

    model_path = cfg['model_path']
    folds = cfg['folds']

    # Load models
    with open(model_path / 'config.yml', 'r') as f:
        cfg_model = yaml.safe_load(f)
            
    models = []
    for ifold in folds:
        model_filename = '%s/model%d.pytorch' % (model_path, ifold)
        model = Model(cfg_model['model'], pretrained=False, verbose=False)
        model.load_state_dict(torch.load(model_filename, weights_only=True))
        model.to(device)
        model.eval()
        models.append(model)

        if process_id == 0:
            print('Load model', model_filename)
        
    # Loop over tomograms
    while not tomo_queue.empty():
        try:
            tomo_path = tomo_queue.get(timeout=1)
            pred = predict(tomo_path, models, cfg)
            pred_queue.put(pred)

        except Empty:
            break
            

# Main

tb = time.time()

# Config
cfg = {
    'model_path': MODEL_PATH,
    'folds': [0,1,2,3,4],
    'batch_size': 16,
    'num_workers': 1,
    'use_amp': True,
    'preprocess_device': 'cuda',
}


#
# List of tomograms
#
tomo_paths = get_tomos(INPUT_PATH, 'test')

if len(tomo_paths) == 3:
    # Some experiment when test is dummy (optional)
    # tomo_paths = get_tomos(INPUT_PATH, 'train', n=20)
    pass

print('Data %d' % len(tomo_paths))

manager = mp.Manager()
tomo_queue = manager.Queue()
pred_queue = manager.Queue()
for tomo_path in tomo_paths:
    tomo_queue.put(tomo_path)

time.sleep(1)
assert not tomo_queue.empty()


#
# Launch process
#
num_processes = 2
tb = time.time()

workers = [mp.Process(target=process_fn,
                      args=(i, tomo_queue, pred_queue, cfg))
           for i in range(num_processes)]

for w in workers:
    w.start()

for w in workers:
    w.join()


dt = time.time() - tb
print('%.2f sec for %d tomos' % (dt, len(tomo_paths)))

# queue to list
preds = []
try:
    while not pred_queue.empty():
        preds.append(pred_queue.get(timeout=1))
except Empty:
    pass

assert len(preds) == len(tomo_paths)


#
# Submit
#
th = 0.5
ofilename = 'submission.csv'
create_submission(preds, th, 'submission.csv')
print(ofilename, 'written')


Data 3
Load model /kaggle/input/bacterial-public/weights/object/baseline/model0.pytorch
Load model /kaggle/input/bacterial-public/weights/object/baseline/model1.pytorch
Load model /kaggle/input/bacterial-public/weights/object/baseline/model2.pytorch
Load model /kaggle/input/bacterial-public/weights/object/baseline/model3.pytorch
Load model /kaggle/input/bacterial-public/weights/object/baseline/model4.pytorch
46.64 sec for 3 tomos
Submit submission.csv: 3 positives / 3 tomo_ids
submission.csv written


In [13]:
! head -n 5 submission.csv
! wc -l submission.csv

tomo_id,Motor axis 0,Motor axis 1,Motor axis 2
tomo_00e047,181,5.40163306e+02,6.06898633e+02
tomo_003acc,396,1.08413574e+03,8.64045178e+02
tomo_01a877,142,6.38296875e+02,2.86794141e+02
4 submission.csv


> The first rule of program optimization: Don't do it.
> 
> The second rule of program optimization: Don't do it yet.
